In [ ]:
import sys
import os
import pandas as pd
sys.path.append('/afs/cern.ch/cms/PPD/PdmV/tools/McM/')
from rest import McM

: 

In [3]:
mcm = McM(dev=False)
#mcm.get('requests', 'PPD-RunIIWinter19PFCalib17pLHE-00001', method='get')
#mcm.get('requests', query='prepid=*-RunIIWinter19PFCalib17pLHE-*&status=new')
results = mcm.get('requests', query='tags=20220516_dekumar_monohbb_2HDMa&approval=define')
len(results)

[2022-11-13 23:38:26,456][INFO] Using SSO cookie file /afs/cern.ch/user/d/dfu/private/mcm-prod-cookie.txt


75

In [7]:
results[1]['time_event']

[30]

In [4]:
undone = []
for r in results:
    r['time_event'] = [30]
    gridpack_path = r['fragment'].split("cms.vstring(\'")[1].split("\')")[0]
    if not os.path.exists(gridpack_path):
        undone.append(gridpack_path)
len(undone)

0

In [9]:
process = []
MH3 = []
MH4 = []
st = []
tb = []
for tokens in undone:
    tokens = tokens.split('_')
    for i in range(len(tokens)):
        if tokens[i].endswith('2HDMa'):
            process.append(tokens[i+1])
        if tokens[i] == 'MH3':
            MH3.append(tokens[i+1])
        if tokens[i] == 'MH4':
            MH4.append(tokens[i+1])
        if tokens[i] == 'sinp':
            st.append(tokens[i+1].replace('p', '.'))
        if tokens[i] == 'tanb':
            tb.append(tokens[i+1].replace('p', '.'))

In [10]:
process, MH3, MH4, st, tb

([], [], [], [], [])

In [93]:
df = pd.DataFrame({'mA': MH3, 'ma': MH4, 'mchi': ['None' for _ in range(len(MH3))], 'tb': tb, 'st': st, 'process': process})
with open('par_scans_2HDMa.txt', 'w') as f:
    df.to_csv(f, sep='\t', index=False, header=True, encoding="utf-8")

In [5]:
for r in results:
    output = mcm.update('requests', r)
    if output['results'] == False:
        print(r['_id'],output['message'])